In [ ]:
import pandas as pd

# Load the dataset, correcting the file path and name
# Assuming the correct file name is "Global_Landslide_Catalog_Export.csv"
glc_data = pd.read_csv("/content/Global_Landslide_Catalog_Export.csv")

# Check the first few rows and column information
print(glc_data.head())
print(glc_data.info())

*This code loads a dataset called "Global_Landslide_Catalog_Export.csv" into a DataFrame using the pandas library, making the data easier to work with in Python. It then displays the first few rows to give an initial view of the data and uses .info() to provide an overview of the dataset’s structure, including column names, data types, and the number of non-missing values. This quick inspection helps us understand the dataset's content and quality before further analysis.*

In [ ]:
# Convert the date column to datetime format
glc_data['event_date'] = pd.to_datetime(glc_data['event_date'], errors='coerce')

# Drop rows with missing dates or any necessary columns
glc_data.dropna(subset=['event_date'], inplace=True)

# Select relevant columns for our model (adjust based on available columns)
relevant_columns = ['event_date', 'latitude', 'longitude', 'landslide_size', 'fatality_count', 'landslide_trigger']
glc_data = glc_data[relevant_columns]

# Check unique triggers to ensure we focus on compound hazards
print(glc_data['landslide_trigger'].unique())

# Filter the dataset for specific triggers related to compound events (e.g., rain, flood, storm)
glc_data = glc_data[glc_data['landslide_trigger'].isin(['rain', 'flood', 'storm'])]

# Fill missing values in critical columns
glc_data.fillna(0, inplace=True)


<ipython-input-13-b17d94c60cc8>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  glc_data['event_date'] = pd.to_datetime(glc_data['event_date'], errors='coerce')


['rain' 'downpour' 'monsoon' 'tropical_cyclone' 'unknown'
 'continuous_rain' 'dam_embankment_collapse' 'no_apparent_trigger' nan
 'other' 'leaking_pipe' 'construction' 'snowfall_snowmelt' 'mining'
 'flooding' 'earthquake' 'freeze_thaw' 'volcano' 'vibration']


*This code processes the landslide dataset by first converting the event_date column to a datetime format, which makes it easier to handle dates. Rows with missing dates are removed to ensure data completeness. We then select only relevant columns—such as date, location, size, fatalities, and trigger type—for analysis. After checking the unique types of landslide triggers, the dataset is filtered to include only events triggered by compound hazards like rain, floods, or storms. Finally, missing values in key columns are filled with zero to prepare the data for analysis or modeling.*

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler


# Convert 'landslide_size' to numerical representation
# Assuming 'landslide_size' has categories like 'small', 'medium', 'large'
size_mapping = {'small': 1, 'medium': 2, 'large': 3}  # Adjust mapping if needed
glc_data['landslide_size'] = glc_data['landslide_size'].map(size_mapping)

# Fill NaN values after mapping, if any
glc_data['landslide_size'].fillna(0, inplace=True) # You might need to adjust fill value

# Initialize the scaler
scaler = StandardScaler()

# Scale the features
glc_data[['latitude', 'longitude', 'landslide_size', 'fatality_count']] = scaler.fit_transform(
    glc_data[['latitude', 'longitude', 'landslide_size', 'fatality_count']]
)

<ipython-input-15-279ced2018e4>:12: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  glc_data['landslide_size'].fillna(0, inplace=True) # You might need to adjust fill value


*This code prepares the glc_data dataset for machine learning by first converting the landslide_size column (which contains categorical values like 'small', 'medium', and 'large') into numerical form using a defined mapping (small = 1, medium = 2, large = 3). If any values are still missing after this mapping, they are replaced with 0. Next, we initialize a StandardScaler to standardize numerical features, ensuring that latitude, longitude, landslide_size, and fatality_count all have a mean of 0 and a standard deviation of 1. This scaling process prepares the data for use in many machine learning algorithms, which perform better with standardized inputs.*

In [ ]:
# Define a target column for compound disaster events
glc_data['compound_event'] = glc_data['landslide_trigger'].apply(
    lambda x: 1 if x in ['rain', 'flood', 'storm'] else 0
)


*This line of code creates a new column called compound_event in the glc_data dataset. The goal is to label landslide events that were caused by compound hazards—specifically, triggers like rain, flood, or storm. Using the apply function, we check each value in the landslide_trigger column, assigning a value of 1 if the trigger is one of these compound events, and 0 otherwise. This binary target column can be useful for classification tasks, helping us distinguish between compound and non-compound disaster events in the dataset.*

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from sklearn.model_selection import train_test_split

# Prepare the feature matrix (X) and target (y)
features = ['latitude', 'longitude', 'landslide_size', 'fatality_count']
X = np.expand_dims(glc_data[features].values, axis=-1)
y = glc_data['compound_event'].values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the LSTM model
model = Sequential([
    LSTM(64, input_shape=(len(features), 1), return_sequences=True),
    Dropout(0.2),
    LSTM(32),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=16, validation_split=0.1)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


117/117 ━━━━━━━━━━━━━━━━━━━━ 5s 10ms/step - accuracy: 0.9357 - loss: 0.4329 - val_accuracy: 1.0000 - val_loss: 1.6683e-04
Epoch 2/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 2.0178e-04 - val_accuracy: 1.0000 - val_loss: 8.9094e-05
Epoch 3/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 1.0000 - loss: 1.2538e-04 - val_accuracy: 1.0000 - val_loss: 5.3348e-05
Epoch 4/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 1.0000 - loss: 8.3922e-05 - val_accuracy: 1.0000 - val_loss: 3.5522e-05
Epoch 5/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 1.0000 - loss: 5.2125e-05 - val_accuracy: 1.0000 - val_loss: 2.5533e-05
Epoch 6/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 1.0000 - loss: 4.7166e-05 - val_accuracy: 1.0000 - val_loss: 1.8619e-05
Epoch 7/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 1.0000 - loss: 3.5521e-05 - val_accuracy: 1.0000 - val_loss: 1.3966e-05
Epoch 8/20
117/117 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accurac

*This code builds and trains an LSTM model to predict whether a landslide event is a compound disaster (caused by rain, flood, or storm) based on features like location, size, and fatalities. The feature matrix `X` is prepared and reshaped to fit the LSTM’s input requirements, while the target `y` represents whether an event is compound. The data is split into training and testing sets, then passed to a sequential LSTM model with dropout layers to reduce overfitting. The model is trained using binary cross-entropy loss and evaluated on the test set, with test accuracy indicating its predictive performance.*

In [ ]:
pip install streamlit  #installing for application


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 5.9 MB/s eta 0:00:00


In [ ]:
# app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# Instead of using __file__, you can directly specify the path to your model
# Assuming the model is in the same directory as the script
model_path = "compound_disaster_model.h5"

# Or use the current working directory:
# model_path = os.path.join(os.getcwd(), "compound_disaster_model.h5")

# Load the trained model and initialize the scaler
# If the model file doesn't exist, display an error message
try:
    model = load_model(model_path)
except FileNotFoundError:
    st.error("Model file not found. Please train and save the model first.")
    st.stop() # Stop execution if model is not found

scaler = StandardScaler()


2024-10-26 15:44:39.115 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-26 15:44:39.202 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-10-26 15:44:39.204 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-26 15:44:39.208 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


*This code sets up a basic Streamlit application to load and use a pre-trained model for predicting compound disaster events. It starts by importing necessary libraries and specifies the path to the saved model file, `"compound_disaster_model.h5"`. The code attempts to load this model using `load_model()`. If the model file is missing, it displays an error message in the Streamlit interface and stops further execution. Additionally, a `StandardScaler` object is initialized to preprocess input data before making predictions with the model.*

In [ ]:
import tensorflow as tf
import pandas as pd
import sklearn
import streamlit

print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")
print(f"Streamlit version: {streamlit.__version__}")

TensorFlow version: 2.17.0
Pandas version: 2.2.2
Scikit-learn version: 1.5.2
Streamlit version: 1.39.0


In [ ]:
# app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

# Instead of using __file__, you can directly specify the path to your model
# Assuming the model is in the same directory as the script
# model_path = "compound_disaster_model.h5"

# Or use the current working directory:
model_path = os.path.join(os.getcwd(), "compound_disaster_model.h5") # Ensure model is in the same directory

# Load the trained model and initialize the scaler
# If the model file doesn't exist, display an error message
try:
    model = load_model(model_path)
except FileNotFoundError:
    st.error(f"Model file not found at path: {model_path}. Please train and save the model first.")
    st.stop() # Stop execution if model is not found

scaler = StandardScaler()

# ... (rest of your code for the Streamlit app)

2024-10-26 15:47:57.019 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-26 15:47:57.022 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-26 15:47:57.025 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
!pip install streamlit
!pip install tensorflow
!pip install pandas
!pip install scikit-learn


In [ ]:
from google.colab import files
uploaded = files.upload()  # This will prompt you to upload the model file.


Saving Global_Landslide_Catalog_Export.csv to Global_Landslide_Catalog_Export (1).csv


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
from tensorflow.keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler
import os

model_path = os.path.join(os.getcwd(), "compound_disaster_model.h5")

try:
    model = load_model(model_path)
except FileNotFoundError:
    st.error(f"Model file not found at path: {model_path}. Please train and save the model first.")
    st.stop()

scaler = StandardScaler()

# Your remaining Streamlit app code goes here.


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501


In [ ]:
!curl -v https://<subdomain>.loca.lt

/bin/bash: line 1: subdomain: No such file or directory


In [ ]:
!npm install -g localtunnel@2.0.2


changed 22 packages, and audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
!npx localtunnel --port 8501 # Replace 8501 with your Streamlit app's port

your url is: https://all-owls-follow.loca.lt
